<a href="https://colab.research.google.com/github/llunasanz/data-project-3/blob/feature%2Fdata_processing/DataProject3LimpiezaDeDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 40.1 MB/s 


In [2]:
pip install sweetviz

     |████████████████████████████████| 15.1 MB 15.3 MB/s 


In [64]:
import pandas as pd
import pyspark as ps
import datetime as dt
import sweetviz as sv
import matplotlib as plt

## CARGA Y LIMPIEZA DE DATOS

### Cargamos los datos de los diferentes CSV para su posterior limpieza de variables






In [32]:
# Usamos pandas para leer de los csv 
ts_perf = pd.read_csv('/content/drive/MyDrive/DP3/data/test_performance.csv')
ts_prev = pd.read_csv('/content/drive/MyDrive/DP3/data/test_previous_loan.csv')
tr_perf = pd.read_csv('/content/drive/MyDrive/DP3/data/train_performance.csv')
tr_prev = pd.read_csv('/content/drive/MyDrive/DP3/data/train_previous_loan.csv')

In [33]:
# Convertimos las cargas en dataframe

df_ts_performance = pd.DataFrame(ts_perf)
df_ts_previous = pd.DataFrame(ts_prev)
df_tr_performance = pd.DataFrame(tr_perf)
df_tr_previous = pd.DataFrame(tr_prev)

#### Limpieza del df test performance

In [59]:
# Eliminamos la columna loannumber

df_ts_performance = df_ts_performance.drop(columns = 'loannumber')

In [60]:
# Categorizamos la columna termdays para distinguir entre short, medium or long y quitamos la variable termdays

uv = df_ts_performance['termdays'].unique()
print(uv)

df_ts_performance['termlength'] = pd.cut(df_ts_performance['termdays'],[0,16,61,89], labels = ['short','medium','large'])

#df_tr_performance.drop(columns = 'termdays')

[30 15 60 90]


In [63]:
# Categorizamos el referredby para que nos de 1 o 0



#FALTA

In [70]:
# Calculamos el tipo de interés impuesto

df_ts_performance['TipoInterésSimple'] = ((df_ts_performance['totaldue'] - df_ts_performance['loanamount']) / (df_ts_performance['loanamount'] * df_ts_performance['termdays']))*100

In [71]:
# Vista del dataframe 
df_ts_performance

,customerid,systemloanid,approveddate,creationdate,loanamount,totaldue,termdays,referredby,termlength,TipoInterésSimple
0,8a858899538ddb8e015390510b321f08,301998974,40:48.0,39:35.0,10000,12250.0,30,NaN,medium,0.750000
1,8a858959537a097401537a4e316e25f7,301963615,43:40.0,42:34.0,40000,44000.0,30,NaN,medium,0.333333
2,8a8589c253ace09b0153af6ba58f1f31,301982236,15:11.0,15:04.0,20000,24500.0,30,NaN,medium,0.750000
3,8a858e095aae82b7015aae86ca1e030b,301971730,00:54.0,00:49.0,30000,34500.0,30,NaN,medium,0.500000
4,8a858e225a28c713015a30db5c48383d,301959177,04:33.0,04:27.0,20000,24500.0,30,NaN,medium,0.750000
...,...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,47:02.0,46:53.0,10000,11500.0,15,NaN,short,1.000000
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,00:45.0,04:05.0,10000,13000.0,30,NaN,medium,1.000000
1447,8a858fde56eb02280156f59b976d46c0,301994653,09:40.0,09:33.0,40000,48000.0,60,NaN,medium,0.333333
1448,8a858e10570f2d65015717fcfec44996,301961483,47:54.0,46:49.0,60000,68100.0,90,NaN,NaN,0.150000


#### Limpieza del df test previous

In [72]:
# Vista del dataframe 
df_ts_previous

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a858899538ddb8e015390510b321f08,301621635,3,2016-05-17 10:37:00.000000,2016-05-17 09:36:55.000000,10000.0,13000.0,30,2016-06-17 00:04:15.000000,NaN,2016-06-16 00:00:00.000000,2016-06-16 15:44:08.000000
1,8a858959537a097401537a4e316e25f7,301810201,5,2017-02-04 21:28:59.000000,2017-02-04 20:28:52.000000,30000.0,36800.0,60,2017-03-02 16:22:58.000000,NaN,2017-03-06 00:00:00.000000,2017-03-02 16:07:47.000000
2,8a858959537a097401537a4e316e25f7,301831255,6,2017-03-04 10:28:22.000000,2017-03-04 09:28:16.000000,30000.0,34400.0,30,2017-04-02 00:44:24.000000,NaN,2017-04-03 00:00:00.000000,2017-04-01 21:29:46.000000
3,8a8589c253ace09b0153af6ba58f1f31,301627292,3,2016-06-02 14:27:14.000000,2016-06-02 13:27:08.000000,10000.0,13000.0,30,2016-07-04 11:34:04.000000,NaN,2016-07-04 00:00:00.000000,2016-07-04 11:19:01.000000
4,8a8589c253ace09b0153af6ba58f1f31,301621095,2,2016-05-16 09:13:12.000000,2016-05-16 08:13:04.000000,10000.0,11500.0,15,2016-06-02 00:02:58.000000,NaN,2016-05-31 00:00:00.000000,2016-06-01 15:22:34.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5902,8a858f085b494826015b54a0b2a1093b,301861709,1,2017-04-09 23:04:32.000000,2017-04-09 22:04:10.000000,10000.0,13000.0,30,2017-05-02 22:27:03.000000,8a858e4f5929c3df0159420645f46f55,2017-05-09 00:00:00.000000,2017-05-02 22:11:54.000000
5903,8a858f565b683b56015b7391c2617960,301869817,1,2017-04-15 23:33:17.000000,2017-04-15 22:33:05.000000,10000.0,11500.0,15,2017-04-25 19:32:42.000000,NaN,2017-05-02 00:00:00.000000,2017-04-25 19:17:33.000000
5904,8a858fab5b9105dc015b975e594e394a,301952446,1,2017-06-26 15:46:28.000000,2017-06-26 14:45:20.000000,10000.0,13000.0,30,2017-07-13 13:12:43.000000,NaN,2017-07-26 00:00:00.000000,2017-07-13 13:02:34.000000
5905,8a858fcf5b39c3ba015b39c7691603e1,301887884,2,2017-05-03 06:56:10.000000,2017-05-03 05:56:04.000000,10000.0,13000.0,30,2017-06-01 22:00:17.000000,NaN,2017-06-02 00:00:00.000000,2017-06-01 21:50:07.000000


In [ ]:
# Limpieza del df train performance

In [ ]:
# Limpieza del df train previous


In [23]:
# Modificamos la columna referredby para que nos diga si si o si no
df_ts_performance['referredby'] = df_ts_performance['referredby'].fillna('0')

In [24]:
df_ts_performance

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby
0,8a858899538ddb8e015390510b321f08,301998974,4,40:48.0,39:35.0,10000,12250.0,30,0
1,8a858959537a097401537a4e316e25f7,301963615,10,43:40.0,42:34.0,40000,44000.0,30,0
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,15:11.0,15:04.0,20000,24500.0,30,0
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,00:54.0,00:49.0,30000,34500.0,30,0
4,8a858e225a28c713015a30db5c48383d,301959177,4,04:33.0,04:27.0,20000,24500.0,30,0
...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,3,47:02.0,46:53.0,10000,11500.0,15,0
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,3,00:45.0,04:05.0,10000,13000.0,30,0
1447,8a858fde56eb02280156f59b976d46c0,301994653,9,09:40.0,09:33.0,40000,48000.0,60,0
1448,8a858e10570f2d65015717fcfec44996,301961483,14,47:54.0,46:49.0,60000,68100.0,90,0




---

---



---



---











Lo descargamos en csv

In [ ]:
train_demografic.to_csv('df_demographic.csv', index = False , sep = ',')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Ahora queremos mergear la de demografic con el performance para sacar la edad


In [ ]:
prev = pd.read_csv('/content/df_previous.csv')

In [ ]:
demo = pd.read_csv('/content/df_demographic.csv')

In [ ]:
demo.head()

,Unnamed: 0,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,Fecha nacimiento
0,0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,Other,Other,Other,1973-10-10
1,1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,Other,Permanent,Other,1986-01-21
2,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,Other,Other,Other,1987-04-01
3,3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,Other,Permanent,Other,1991-07-19
4,4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,Other,Permanent,Other,1982-11-22


In [ ]:
df_prev_demo = pd.merge(demo, prev, on='customerid')

In [ ]:
df_prev_demo.head()

,Unnamed: 0,customerid,birthdate_x,bank_account_type_x,longitude_gps_x,latitude_gps_x,bank_name_clients_x,bank_branch_clients_x,employment_status_clients_x,level_of_education_clients_x,...,firstduedate,firstrepaiddate,birthdate_y,bank_account_type_y,longitude_gps_y,latitude_gps_y,bank_name_clients_y,bank_branch_clients_y,employment_status_clients_y,level_of_education_clients_y
0,0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,Other,Other,Other,...,"2017-07-05 00:00:00,000","2017-07-04 17:59:36,000","1973-10-10 00:00:00,000",Savings,"3,319219","6,5286039",GT Bank,Other,Other,Other
1,1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,Other,Permanent,Other,...,"2017-07-07 00:00:00,000","2017-07-10 08:42:44,000","1986-01-21 00:00:00,000",Savings,"3,3255983","7,1194033",Sterling Bank,Other,Permanent,Other
2,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,Other,Other,Other,...,"2017-07-10 00:00:00,000","2017-07-11 10:02:11,000","1987-04-01 00:00:00,000",Savings,"5,7460998","5,5631737",Fidelity Bank,Other,Other,Other
3,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,Other,Other,Other,...,"2017-05-23 00:00:00,000","2017-05-27 12:52:45,000","1987-04-01 00:00:00,000",Savings,"5,7460998","5,5631737",Fidelity Bank,Other,Other,Other
4,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,Other,Other,Other,...,"2017-06-12 00:00:00,000","2017-06-08 11:03:40,000","1987-04-01 00:00:00,000",Savings,"5,7460998","5,5631737",Fidelity Bank,Other,Other,Other
